In [1]:
import json
import nltk
import re
import string
from nltk.corpus import stopwords 
import demoji
from nltk.chunk import conlltags2tree, tree2conlltags
import spacy
from imdb import IMDb, IMDbError
import itertools
stop_words = set(stopwords.words('english')) 
# demoji.download_codes()

In [2]:
with open('gg2020.json', encoding='utf8') as json_file:
    data = [json.loads(line) for line in json_file]

In [3]:
data = json.load(open('gg2013.json'))

In [4]:
print(string.punctuation)
newstr_punct = string.punctuation
newstr_punct = newstr_punct.translate({ord('-'): None})
print(newstr_punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&'()*+,./:;<=>?@[\]^_`{|}~


In [5]:
def strip_all_entities(text):
    entity_prefixes = ['@','#']
    for separator in newstr_punct:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

#### award

In [6]:
clean_data = []
for tweet in data:
    tweet_text = tweet['text']
    no_http = re.sub('http://\S+|https://\S+', '', tweet_text)
    remove_tag = strip_all_entities(no_http)
    clean_data.append(remove_tag)
clean_data.sort()
clean_data = list(set(clean_data))

In [121]:
start_pattern = re.compile(r'w[oi]ns? best ')
start_pattern2 = re.compile(r'[Bb]est [Pp]erformance ')
start_pattern3 = re.compile(r'[Bb]est [Aa]ct')
start_pattern4 = re.compile(r'[Bb]est [Ss]upport')

In [122]:
best_split = []
for i in clean_data:
    if len(re.findall(r'w[oi]ns? best ', i)) > 0:
        split = start_pattern.split(i)
        if len(split) > 1:
            best_split.append('best ' + split[1])
    elif len(re.findall(r'[Bb]est [Pp]erformance ', i)) > 0:
        split = start_pattern2.split(i)
        if len(split) > 1:
            best_split.append('best performance ' + split[1])
    elif len(re.findall(r'[Bb]est [Aa]ct', i)) > 0:
        split = start_pattern3.split(i)
        if len(split) > 1:
            best_split.append('best act' + split[1])
    else:
        split = start_pattern4.split(i)
        if len(split) > 1:
            best_split.append('best support' + split[1]) 

In [123]:
for item in clean_data:
    if len(re.findall(r'Harris', item)) > 0:
        print(item)

Best Supporting Actor in a TV Series Ed Harris Game Change
Et le pour le meilleur acteur de soutien dans un série télé minisérie va à Ed Harris
RT Best Supporting Actor in Mini Series TV Movie Ed Harris Game Change
Ed Harris wins Best Supporting Actor TV Movie for Game Change in absentia
Best Supporting Actor TV Ed Harris Game Change
I know Me too “@MHarrisPerry There was simply not enough Tina and Amy Watched the whole thing waiting for more
Best supporting actor TV miniseries series TV movie whew Ed Harris Game Change
Loved this lightening fast reaction zing “@MHarrisPerry Glad we got you through Middle School
Mejor actor de reparto en una serie miniseries o película para TV Ed Harris - Game Change
Ed Harris Try Ed I-Have-No-Harris
RT Mejor Actor de Reparto serie o película de TV Ed Harris por Game Change
RT Best Supporting Actor in a Television Series Ed Harris Game Change
I should say Silence of the Lambs era Clarice is my hero Thomas Harris you re still on notice for Hannibal
RT W

In [38]:
best_split2 = [] 
for item in best_split:
    tmp_item = item.lower()
    word_list = tmp_item.split()
    word_len = len(word_list)
    for idx in range(word_len):
        true_dix = word_len - 1 - idx
        if word_list[true_dix] not in awards_end_words1:
            word_list.pop(true_dix)
        else:
            break
    if len(word_list) > 2:
        best_split2.append(' '.join(word for word in word_list))

In [113]:
best_split3 = []
for item in best_split2:
    new_item = re.sub(r't.v.','television',item)
    new_item = re.sub(r'tv','television',item)
    new_item = re.sub(r'best act','best performance by an act',new_item)
    new_item = re.sub(r'best supporting actor','best performance by an actor in a supporting role',new_item)
    new_item = re.sub(r'best supporting actress','best performance by an actress in a supporting role',new_item)
    new_item = re.sub(r'mini series','mini-series',new_item)
    new_item = re.sub(r'miniseries','mini-seiries',new_item)
    if 'picture drama' in new_item:
        new_item = re.sub(r'picture drama','picture - drama',new_item)
    if 'picture comedy' in new_item:
        new_item = re.sub(r'picture comedy','picture - comedy',new_item)
    if 'supporting' in new_item and 'series' in new_item:
        #print(new_item)
        word_list = new_item.split()
        word_len = len(word_list)
        for idx in range(word_len):
            true_dix = word_len - 1 - idx
            if word_list[true_dix] != 'role':
                word_list.pop(true_dix)
            else:
                break
        if len(word_list) > 1:
            best_split3.append(' '.join(word for word in word_list) + ' in a series, mini-series or motion picture made for television')
    elif 'supporting' not in new_item and  'mini-series' in new_item:
        word_list = new_item.split()
        word_len = len(word_list)
        for idx in range(word_len):
            true_dix = word_len - 1 - idx
            if word_list[true_dix] != 'in':
                word_list.pop(true_dix)
            else:
                break
        if len(word_list) > 1:
            best_split3.append(' '.join(word for word in word_list) + ' a mini-series or motion picture made for television')
    else:
        best_split3.append(new_item)

In [117]:
valid_awards = [a[0] for a in nltk.FreqDist(best_split3).most_common(300)]
for i in valid_awards:
    word_list = i.split()
    if len(word_list) >= 7:
        print(i)

best performance by an actress in a supporting role in a motion picture
best performance by an actor in a motion picture - drama
best performance by an actress in a motion picture - drama
best performance by an actor in a supporting role in a motion picture
best performance by an actor in a motion picture - comedy or musical
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a comedy
best performance by an actress in a television series drama
best performance by an actress in a motion picture - comedy or musical
best performance by an actress in a television series - drama
best performance by an actress in a mini-series or motion picture made for television
best performance by an actress in a comedy or musical
best performance by an actor in a television series drama
best performance by an actor in a supporting role in a television
best performance by an actor in a television
best performa

In [118]:
awards_end_words2 = ['drama', 'musical', 'comedy', 'picture',  'miniseries', 'mini-series', 'television', 'tv', 't.v.', 'film']

In [114]:
end_pattern = re.compile(r' for')
end_pattern2 = re.compile(r' at')
#end_pattern3 = re.compile(r' goes to')
#end_pattern4 = re.compile(r' is')
#end_pattern5 = re.compile(r' [Gg]olden')
#end_pattern6 = re.compile(r' [Tt]he ')
#end_pattern7 = re.compile(r' this')
#end_pattern8 = re.compile(r' answer')
#end_pattern9 = re.compile(r' -')
#end_pattern10 = re.compile(r' well')

In [115]:
for item in best_split3:
    if 'supporting' not in item and 'act' in item and 'mini-series' in item:
        print(item)

best performance by an actor in a mini-series or motion picture made for television
best performance by an actor in a mini-series or motion picture made for television
best performance by an actor in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture made for television
best performance by an actor in a mini-series or motion picture made for television
best performance by an actor in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture made for television
best performance by an actor in a mini-series or motion picture made for television
best performance by an actress in a mini-series or motion picture 

In [137]:
awards_end_words2 = ['drama', 'musical', 'comedy', 'picture',  'miniseries', 'mini-series', 'television', 'tv', 't.v.', 'film']
start_pattern = re.compile(r'[Bb]est ')


In [138]:
best_split_film = []
for i in clean_data:
    if len(re.findall(r'[Aa]ct', i)) > 0:
        continue
    if len(re.findall(r'w[oi]ns? best ', i)) > 0:
        split = start_pattern.split(i)
        if len(split) > 1:
            best_split_film.append('best ' + split[1])

In [139]:
best_split_film2 = [] 
for item in best_split_film:
    tmp_item = item.lower()
    word_list = tmp_item.split()
    word_len = len(word_list)
    for idx in range(word_len):
        true_dix = word_len - 1 - idx
        if word_list[true_dix] not in awards_end_words2:
            word_list.pop(true_dix)
        else:
            break
    if len(word_list) > 2:
        best_split_film2.append(' '.join(word for word in word_list))

In [140]:
valid_awards = [a[0] for a in nltk.FreqDist(best_split_film2).most_common(300)]
for i in valid_awards:
    word_list = i.split()
    print(i)

best motion picture
best tv drama
best picture drama
best foreign film
best animated film
best picture - drama
best original song in a motion picture
best picture musical or comedy
best original song for motion picture
best motion picture comedy or musical
best picture - comedy or musical
best comedy musical
best tv series drama
best tv series - comedy or musical
best mini-series or tv
best tv comedy
best foreign language film
best picture comedy or musical
best animated feature film
best comedy or musical tv
best show on tv
best motion picture drama
best tv series musical or comedy
best drama tv
best director for the argo film
best screenplay motion picture
best blurry picture
best musical or comedy
best picture but i think zero dark thirty was a better film
best picture i could see it 100 times glorious movie and musical
best tv miniseries
best song in a motion picture
best musical picture
best drama globes what a pleasant surprise much deserved recognition for ben afleck s film
best

In [259]:
best_split_end = []
for i in best_split:
    if len(re.findall(r' for', i)) > 0:
        split = end_pattern.split(i)
        if len(split) > 1:
            if len(split[0]) > len('best'):
                best_split_end.append(split[0])
    elif len(re.findall(r' at', i)) > 0:
        split = end_pattern2.split(i)
        if len(split) > 1:
            if len(split[0]) > len('best'):
                best_split_end.append(split[0])
    else:
        best_split_end.append(i)

In [260]:
best_split_end2 = []
for i in best_split_end:
    if len(re.findall(r' goes to', i)) > 0:
        split = end_pattern3.split(i)
        if len(split) > 1:
            if len(split[0]) > len('best'):
                best_split_end2.append(split[0])
    elif len(re.findall(r' is', i)) > 0:
        split = end_pattern4.split(i)
        if len(split) > 1:
            if len(split[0]) > len('best'):
                best_split_end2.append(split[0])
    #elif len(re.findall(r' [Tt]he ', i)) > 0:
    #    split = end_pattern6.split(i)
    #    if len(split) > 1:
    #        if len(split[0]) > len('best'):
    #            best_split_end2.append(split[0])
    #elif len(re.findall(r' [Gg]olden', i)) > 0:
    #    split = end_pattern5.split(i)
    #    if len(split) > 1:
    #        if len(split[0]) > len('best'):
    #            best_split_end2.append(split[0])
    else:
        best_split_end2.append(i)

In [261]:
best_split3 = []
for item in list(best_split_end):
    if len(re.findall(r'[Bb]est [Pp]erformance by an', item)) > 0 and len(re.findall(r'[Aa]ct', item)) > 0 and len(re.findall(r' in a ', item)) > 0:
        new_item = re.sub(r'[Tt][Vv]','television',item)
        new_item = re.sub(r'[Mm]ade','made for television',new_item)
        new_item = re.sub(r'[Mm]ini[Ss]eries','mini-series',new_item)
        new_item = re.sub(r'[Dd]rama.*$','drama',new_item)
        new_item = re.sub(r'[Mm]usical.*$','musical',new_item)
        new_item = re.sub(r'[Mm]usic.*$','musical',new_item)
        new_item = re.sub(r' any ',' a ',new_item)
        #new_item = re.sub(r' [Tt]his.*$','',new_item)
        #new_item = re.sub(r' [Aa]nswer.*$','',new_item)
        new_item = re.sub(r'—','-',new_item)
        new_item = re.sub(r'–','-',new_item)
        #new_item = re.sub(r'-',' - ',new_item)
        #new_item = re.sub(r'  ',' ',new_item)
        best_split3.append(new_item.lower())
best_split4 = []
for item in list(best_split3):
    if 'musical or comedy' in item:
        new_item = re.sub(r'musical or comedy','comedy or musical',item)
    elif 'comedy' in item and 'musical' not in item:
        new_item = re.sub(r'comedy','comedy or musical',item)
    elif 'comedy' not in item and 'musical' in item:
        new_item = re.sub(r'musical','comedy or musical',item)
    else:
        new_item = item

    best_split4.append(new_item)

best_split5 = []
for item in list(best_split4):
    if  'motion picture' in item and 'motion picture ' not in item:
        continue
    if  'drama' in item and '- drama' not in item:
        new_item = re.sub(r'drama','- drama',item)
    elif 'comedy' in item and '- comedy' not in item:
        new_item = re.sub(r'comedy','- comedy',item)
    #elif 'this' in item:
    #    split = end_pattern7.split(i)
    #    if len(split) > 1:
    #        new_item = split[0]
    #elif 'answer' in item:
    #    split = end_pattern8.split(i)
    #    if len(split) > 1:
    #        new_item = split[0]
    elif 'supporting role' in item and '-' in item:
        split = end_pattern9.split(i)
        if len(split) > 1:
            new_item = split[0]
    #elif 'well' in item:
    #    split = end_pattern10.split(i)
    #    if len(split) > 1:
    #        new_item = split[0]
    else:
        new_item = item

    best_split5.append(new_item)

#best_split6 = []
#for item in list(best_split5):
#    word_list = item.split()
#    print(word_list)
 #   for word in reverse(word_list):
#for item in list(best_split5):
#    print(item)

In [111]:
valid_awards = [a[0] for a in nltk.FreqDist(best_split3).most_common(300)]

In [112]:
for i in valid_awards:
    print(i)

best performance by an actress in a supporting role in a motion picture
best performance by an actor in a supporting role in a motion picture
best performance by an actor in a motion picture drama
best performance by an actress in a motion picture drama
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a comedy
best performance by an actor in a motion picture - drama
best performance by an actress in a television series drama
best performance by an actor drama
best performance by an actress in a television series - drama
best performance by an actress in a mini-series or motion picture made for television
best performance by an actress drama
best performance by an actress in a comedy or musical
best performance by an actor in a motion picture - comedy or musical
best performance by an actor in a television series drama
best performance by an actress in a motion picture - drama
best perfor

In [161]:
re.sub(r'  ',' ','x  -x')

'x -x'

#### others

In [65]:
awards = []
pattern = re.compile(r'(?<=w[oi]n[s]\s)[Bb]est.*')
tweet_sep = ['for','who',' s ','made','yet','?', 'golden', 'globe', 'globes',' | ', 'with' ,' at', ' http', ' #', '(','.', ',', '!', '?','\\', ':', ';', '"', "'",'the','but','although', 'made']
awards_bag_of_words = ['-','performance', 'actress', 'actor', 'supporting', 'role', 'director', 'motion', 'picture', 'drama','animated', 'feature', 'film', 'song','comedy', 'musical', 'language', 'foreign','screenplay', 'orginal', 'television', 'tv', 'series', 'mini-series', 'mini']

In [67]:
for x in range(1):
    for tweetx in data:
        tweet = tweetx['text']
        #print(tweet)
        award = re.findall(pattern, tweet)
        if award:
            for a in award:
                a = a.lower()
                for sep in tweet_sep:
                    if sep in a:
                        a = a.split(sep)[0]
                if "/" in a:
                    a = a.replace('/',' or ')
                awards.append(a)

In [69]:
print(awards)

['best personality on ', 'best movie', 'best dressed', 'best facial hair of ', 'best supporting actor ', 'best supporting actor', 'best supporting', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor', 'best supporting actor', 'best supporting actor ', 'best suppporting actor', 'best supporting actor ', 'best supporting actor ', 'best supporting actor', 'best supporting actor ', 'best supporting actor', 'best supporting actor - motion picture ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actor ', 'best supporting actress in a miniseries ', 'best supporting actor', 'best supporting actress in a miniseries ', 'best supporting actor', 'best supporting actress in a tv show', 'best director', 'best supporting actor ', 'best supporting actor', 'best supporting actor', 'best supporting acto

In [68]:
for x in range(1):
    valid_awards = []
    for each in awards:
        each = each.lower()
        if len(set(each.split(" ")).intersection(set(awards_bag_of_words)))>2:
            valid_awards.append(each)
    print(valid_awards)
    valid_awards = [a[0] for a in nltk.FreqDist(valid_awards).most_common(150)]
    valid_awards.sort(key=lambda a: len(a),reverse=True)
    valid_awards = remove_duplicate(valid_awards, 90)
    valid_awards = dataProcess(valid_awards)

['best supporting actor - motion picture ', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best supporting tv actress in downton abbey', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best supporting actress in a tv show', 'best mini-series or motion picture ', 'best mini series or  tv movie', 'best supporting actress in a tv show', 'best tv mini series or drama', 'best mini series or tv movie', 'best supporting actress in a tv show', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or motion picture ', 'best actress in a tv series ', 'best actress in mini-series or tv movie ', 'best actress in a mini-series or tv movie ', 'best actress in a miniseries or motion picture ', 'best actress in a mini series or tv movie', 'best actress in a mini-series or tv movie ', 'best tv mini series or drama', 'best actress in a miniseries or motion picture ', 'best actress in a mini-series or tv movie ', 'be

NameError: name 'remove_duplicate' is not defined